# Upwind Scheme

In the previous numerical integration of the advection equation, we worked with a numerical scheme that is forward in time and backwards in space. There are several finite difference schemes, some of which are particularly named, such as the [leapfrog](https://en.wikipedia.org/wiki/Leapfrog_integration) or [upwind](https://en.wikipedia.org/wiki/Upwind_scheme) schemes. Here, we'll approach the advection equation again employing the upwind finite difference scheme.

Again, our one-dimensional linear advection equation is 

$$ \frac{\partial \psi}{\partial t} = - u \frac{\partial \psi}{\partial t} .$$

## Setting Up

First, let's import some modules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Now, let's define our grid. Let's also change our initial disturbance to no longer be a cosine wave but rather a Gaussian distribution.

In [ ]:
# Define the domain length, number of grid points, and grid spacing
L = 20.
nx = 300
dx = L/(nx-1)

# Create the x grid
x = np.linspace(-L/2., L/2., nx)

# Create the intial disturbance
psi = np.exp(-(x+6)**2)

# Plot the disturbance
fig,ax = plt.subplots(figsize=(12,6));
ax.plot(x,psi);
ax.set_title("Intial disturbance, $\psi(x,t=0)$",fontsize=18);
ax.set_xlabel("x");
ax.set_ylabel("y");
ax.set_xlim([-L/2.,L/2.]);

### Stability

The Wikipedia article linked above for the upwind scheme refers to *stability* and the Courant–Friedrichs–Lewy (CFL) condition which is

$$ \Big| \frac{u \Delta t}{\Delta x} \Big| \leq 1 .$$

Basically, if this condition is broken, the advection model will not work properly. We'll show an example later where we choose poorly and violate this condition. For now, note that we have define $u$ and $\Delta x$, so we ought to fit $\Delta t$ within this condition. It's good practice to make sure we keep this number quite low, so let's use

$$ \Big| \frac{u \Delta t}{\Delta x} \Big| = 0.1 $$

to solve to $\Delta t$. Doing so, we get

$$ \Delta t = \frac{0.1 \Delta x}{u} .$$

**A quick note**: this has some implications for numerical computations. The wave in the previous wave sections looked pretty blocky, so this time I chose to resolve more of the disturbance by increasing `nx`, simply to make a more pleasant looking plot. However, since

$$ \Delta x = \frac{L}{n_x-1}, $$

this decreased $\Delta x$. Now, to satisfy the CFL criteria, $\Delta t$ is proportional to $\Delta x$, and since I've chosen to make the disturbance look less clunky, I've  indadvertantly reduced my timestep. This means I will need to run the model for longer if I wish to forecast the equation for longer. Now, this is okay for such a simple equation and small, one-dimensional domain since it's going to compute fairly quickly, but when the arrays being calculated are larger and multi-dimensional, we either need to suffer by increasing our gridspacing or we may be forced to find better computation techniques or simply use more advanced hardware. However, this isn't always possible.

Let's go ahead and implement the CFL solution for $\Delta t$ and see the result.

In [ ]:
# Set the time step in accordance with the CFL criteria
dt = 0.1*dx/u
print(f"Our stable choice of dt is: {round(dt,5)}")

# Stop after ~20 seconds of advecting the wave
stop_time = 20.
nt = round(stop_time/dt)+1

## Implementing the Upwind Scheme

Okay, so what is the upwind scheme, and where does it come into play? Our model is going to look a lot like it did before, but with one slight difference in the `advection` function from the previous exercise.

Assuming the disturbance moves at all, according the advection equation, it will only move either left or right depending on the direction of the flow, $u$. Let's assume $u$ is positive, so the flow will advect the wave to the right, or eastward. Consider an arbitrary point $i$ on the grid. If the flow, or the *wind* is blowing eastward, the position $i-1$ is "upwind" while the position $i+1$ is "downwind". The case is opposite for $u<0$; $i-1$ is downwind while $i+1$ is upwind. If the finite differencing scheme has more points on the upwind side, it is referred to as an upwind biased scheme, or an upwind scheme for short.

We could possibly program two different functions depending on the sign of $u$. If $u>0$, the first order upwind scheme would be given by the finite difference approximation

$$ 
\frac{\psi_i^{(n+1)} - \psi_i^{(n)}}{\Delta t} = - u \frac{\psi^{(n)}_{i} - \psi^{(n)}_{i-1}}{\Delta x} \\
\psi_i^{(n+1)} - \psi_i^{(n)} = \Delta t \Big( - u \frac{\psi^{(n)}_{i} - \psi^{(n)}_{i-1}}{\Delta x} \Big) \\
\psi_i^{(n+1)} = \psi_i^{(n)} - u \Delta t \frac{\psi^{(n)}_{i} - \psi^{(n)}_{i-1}}{\Delta x} .
$$
   
On the other hand, if $u<0$, the first order upwind scheme would be

$$ 
\frac{\psi_i^{(n+1)} - \psi_i^{(n)}}{\Delta t} = - u \frac{\psi^{(n)}_{i+1} - \psi^{(n)}_{i}}{\Delta x} \\
\psi_i^{(n+1)} - \psi_i^{(n)} = \Delta t \Big( - u \frac{\psi^{(n)}_{i+1} - \psi^{(n)}_{i}}{\Delta x} \Big) \\
\psi_i^{(n+1)} = \psi_i^{(n)} - u \Delta t \frac{\psi^{(n)}_{i+1} - \psi^{(n)}_{i}}{\Delta x} .
$$
   
Rather than coding two separate functions though, we can combine them, as such:

$$ 
\psi_i^{(n+1)} = \psi_i^{(n)} - \Delta t \Big( \big( \max(u,0) \frac{\psi^{(n)}_{i} - \psi^{(n)}_{i-1}}{\Delta x} \big) + \big( \min(u,0) \frac{\psi^{(n)}_{i+1} - \psi^{(n)}_{i}}{\Delta x} \big) \Big) .
$$

If $u>0$, $\min(u,0) = 0$ and $\max(u,0) = u$, so the equation above becomes

$$ 
\psi_i^{(n+1)} = \psi_i^{(n)} - u \Delta t \frac{\psi^{(n)}_{i} - \psi^{(n)}_{i-1}}{\Delta x}  .
$$

Likewise, if $u<0$, $\min(u,0) = u$ and $\max(u,0) = 0$, and the equation above becomes

$$ 
\psi_i^{(n+1)} = \psi_i^{(n)} - u \Delta t \frac{\psi^{(n)}_{i+1} - \psi^{(n)}_{i}}{\Delta x} .
$$

To make this more compact, we define the following

$$ 
\begin{align}
u^+ & = \max(u,0) \\
u^- & = \min(u,0) \\
\psi_x^+ & = \frac{\psi^{(n)}_{i+1} - \psi^{(n)}_{i}}{\Delta x} \\
\psi_x^- & = \frac{\psi^{(n)}_{i} - \psi^{(n)}_{i-1}}{\Delta x}
\end{align} 
$$

to simplify the upwind scheme for our advection equation to be 

$$ \psi_i^{(n+1)} = \psi_i^{(n)} - \Delta t \big( u^+ \psi_x^- + u^- \psi_x^+ \big) .$$

Let's try to implement this in our advection code. Let's create the functions we'll need to use first, similar to what we did before.

In [ ]:
def copy_functions(psi1):
    
    '''Copy one function to another'''
        
    # Copy psi1 to psi2
    psi2 = np.copy(psi1)
    
    return psi2

def upwind_advection(psi_future,psi_present,u,dt,nx,dx,boundary_condition="periodic"):

    '''Update the wave function via the advection equation'''

    # Define the four upwind parameters
    u_plus = max(u,0)
    u_minus = min(u,0)
    psi_plus = copy_functions(psi_present)
    psi_minus = copy_functions(psi_present)
    psi_plus[:-1] = (psi_present[1:]-psi_present[:-1])/dx
    psi_minus[1:] = (psi_present[1:]-psi_present[:-1])/dx
        
    # Boundary Conditions
    if boundary_condition == "periodic":
        psi_plus[-1] = (psi_present[0]-psi_present[-1])/dx
        psi_minus[0] = (psi_present[0]-psi_present[-1])/dx
    else:
        print("Chosen boundary condition not yet implemented!")
        
    # Update according to advection equation
    psi_future = psi_present - dt*(u_plus*psi_minus + u_minus*psi_plus)
    
    return psi_future

def plot_function(x,psi,time,u,x0):
    
    '''Plot the function for time t'''

    # Plot function
    fig,ax = plt.subplots(figsize=(12,6));
    ax.plot(x,psi_future,label="Predicted");
    ax.plot(x,np.exp(-(x-x0-((u*time)))**2),label="Actual")
    ax.set_xlabel("$x$",fontsize=14);
    ax.set_ylabel("$\psi$",fontsize=14);
    ax.set_title(f"t = {round(time,1)}",fontsize=18);
    ax.set_xlim(min(x),max(x))
    ax.legend(fontsize=14,loc='upper right')

Note that we had to implement the boundary conditions within model integration directly. This is because our $\psi_x^+$ and $\psi_x^-$ need different boundary conditions since the differencing direction is different for either easterly or westerly flow.

Also, in the `plot_function`, I have set the function to plot two curves; the predicted function from the model integration and the actual solution. Since our function is a Gaussian, and the flow is advecting the wave by the amount

$$ \delta x = ut ,$$

the actual solution will be simply shifted by the amount $\delta x$, or

$$ \psi(x,t) = \exp((x - x_0 - \delta x)^2) = \exp((x - x_0 - ut)^2) .$$

By plotting these two side by size, we can see how our model is performing. This exact solution doesn't take into account our boundary condition without some additional configuration, so I made sure to set `nt` to be small enough to stop the model from having the disturnance hit the boundary.

## Westerly Flow Example

Now let's implement our model with a constant eastward flow of $u=0.5$.

In [ ]:
# Define the constant flow velocity
u = 0.5

# Set up the initial conditions
psi_future = copy_functions(psi)
psi_future[-1] = psi_future[0]

# Plot the initial conditions
plot_function(x,psi_future,0.,u,-6)

# Loop through times and update the wave function
for n in range(1,nt+1):
    
    # Advance n to n+1
    psi_present = copy_functions(psi_future)
        
    # Update with advection
    psi_future = upwind_advection(psi_future,psi_present,u,dt,nx,dx)
    
    # PLot the function
    if n % round(nt/4) == 0:
        plot_function(x,psi_future,n*dt,u,-6)

We can see the model is advecting the disturbance eastward, but the amplitude is dropping. In particular, it seems like the model is smoothing out the zonal gradient. Let's try doing this again, but decreasing the gridspacing. We'll keep the same domain and model run time.

In [ ]:
# Define the domain length, number of grid points, and grid spacing
nx = 3000
dx = L/(nx-1)

# Create the x grid
x = np.linspace(-L/2., L/2., nx)

# Create the intial disturbance
psi = np.exp(-(x+6)**2)

# Set the time step in accordance with the CFL criteria
dt = 0.1*dx/u
print(f"Our stable choice of dt is: {round(dt,5)}")

# Stop after ~20 seconds of advecting the wave
stop_time = 20.
nt = round(stop_time/dt)+1

# Set up the initial conditions
psi_future = copy_functions(psi)
psi_future[-1] = psi_future[0]

# Plot the initial conditions
plot_function(x,psi_future,0.,u,-6)

# Loop through times and update the wave function
for n in range(1,nt+1):
    
    # Advance n to n+1
    psi_present = copy_functions(psi_future)
        
    # Update with advection
    psi_future = upwind_advection(psi_future,psi_present,u,dt,nx,dx)
    
    # PLot the function
    if n % round(nt/4) == 0:
        plot_function(x,psi_future,n*dt,u,-6)

So now we see the model is more accurate for smaller $\Delta x$. By increasing the number of points tenfold over the same width, we decreased the size of the gridspacing to one tenth of it's original value. Not surprisingly, the model performs better for smaller gridspacing. This is due to small errors introduced by our first order finite differencing. The definition

$$ \frac{\partial \psi}{\partial x} = \frac{\psi^{(n)}_{i+1} - \psi^{(n)}_{i}}{\Delta x} $$

is really only valid for infinitesimally small $\Delta x$, from the definition of a derivative.

## Easterly Flow Example

Now, let's look at the opposite flow case, $u = -0.5$. This time, the opposite differencing scheme should be chosen. We'll also move the disturbance to the other side.

In [ ]:
# Define the domain length, number of grid points, and grid spacing
nx = 3000
dx = L/(nx-1)

# Set the flow to be easterly
u = -0.5

# Create the x grid
x = np.linspace(-L/2., L/2., nx)

# Create the intial disturbance
psi = np.exp(-(x-6)**2)

# Set the time step in accordance with the CFL criteria
dt = 0.1*dx/abs(u)
print(f"Our stable choice of dt is: {round(dt,5)}")

# Stop after ~20 seconds of advecting the wave
stop_time = 20.
nt = round(stop_time/dt)+1

# Set up the initial conditions
psi_future = copy_functions(psi)
psi_future[-1] = psi_future[0]

# Plot the initial conditions
plot_function(x,psi_future,0.,u,6)

# Loop through times and update the wave function
for n in range(1,nt+1):
    
    # Advance n to n+1
    psi_present = copy_functions(psi_future)
        
    # Update with advection
    psi_future = upwind_advection(psi_future,psi_present,u,dt,nx,dx)
    
    # PLot the function
    if n % round(nt/4) == 0:
        plot_function(x,psi_future,n*dt,u,6)

## FTBS Implementation with Easterly Flow

Let's try this example now going back to our forward in time, backwards in space (FTBS) method. We'll bring back the functions `advection` and `boundary_condition` from the previous section, and then modify the plotting  function to just show a more accurate time stamp. For now, I'll also switch back to the original `nx = 300` choice that gave us the dampening disturbance over time.

In [ ]:
def advection(psi_future,psi_present,u,dt,nx,dx):

    '''Update the wave function via the advection equation'''

    # Update according to advection equation
    psi_future[1:nx] = psi_present[1:nx] - (u*dt*(psi_present[1:nx]-psi_present[0:nx-1])/dx)
    
    return psi_future

def boundary_condition(psi,nx):
    
    '''Apply a periodic boundary condition'''
    
    # Apply the periodic boundary condition to the left side
    psi[0] = psi[nx-1]
    
    return psi

def plot_function(x,psi,time,u,x0):
    
    '''Plot the function for time t'''

    # Plot function
    fig,ax = plt.subplots(figsize=(12,6));
    ax.plot(x,psi_future,label="Predicted");
    ax.plot(x,np.exp(-(x-x0-((u*time)))**2),label="Actual")
    ax.set_xlabel("$x$",fontsize=14);
    ax.set_ylabel("$\psi$",fontsize=14);
    ax.set_title(f"t = {round(time,2)}",fontsize=18);
    ax.set_xlim(min(x),max(x))
    ax.legend(fontsize=14,loc='upper right')
    
# Define the domain length, number of grid points, and grid spacing
nx = 300
dx = L/(nx-1)

# Set the flow to be easterly
u = -0.5

# Create the x grid
x = np.linspace(-L/2., L/2., nx)

# Create the intial disturbance
psi = np.exp(-(x)**2)

# Set the time step in accordance with the CFL criteria
dt = 0.1*dx/abs(u)
print(f"Our stable choice of dt is: {round(dt,5)}")

# Specify the number of time steps
nt = 300

# Set up the initial conditions
psi_future = copy_functions(psi)
psi_future[-1] = psi_future[0]

# Plot the initial conditions
plot_function(x,psi_future,0.,u,0)

# Loop through times and update the wave function
for n in range(1,nt+1):
    
    # Advance n to n+1
    psi_present = copy_functions(psi_future)
        
    # Update with advection
    psi_future = advection(psi_future,psi_present,u,dt,nx,dx)
    
    # Apply boundary conditions
    psi_future = boundary_condition(psi_future,nx)
    
    # PLot the function
    if n in [20,40,60,80,100,200,300]:
        plot_function(x,psi_future,n*dt,u,0)

Our model is blowing up, this time! This choice, using backwards finite differencing with a westward propagating flow, is performing the finite differencing *downwind*. All of our implementation previously has been upwind and hasn't had this problem, but it turns out that downwind schemes are unconditionally unstable. 

## Downwind Scheme

The other downwind option is to perform FTFS (forward in time, forward in space) for $u>0$. Let's go ahead and create a `downwind_advection` scheme and check out the case for westerly flow. All we need to do is switch the implementation so that we have

$$ \psi_i^{(n+1)} = \psi_i^{(n)} - \Delta t \big( u^- \psi_x^- + u^+ \psi_x^+ \big) $$

and we'll have a permenantly broken, downwind scheme.

In [ ]:
def downwind_advection(psi_future,psi_present,u,dt,nx,dx,boundary_condition="periodic"):

    '''Update the wave function via the advection equation'''

    # Define the four downwind parameters
    u_plus = max(u,0)
    u_minus = min(u,0)
    psi_plus = copy_functions(psi_present)
    psi_minus = copy_functions(psi_present)
    psi_plus[:-1] = (psi_present[1:]-psi_present[:-1])/dx
    psi_minus[1:] = (psi_present[1:]-psi_present[:-1])/dx
        
    # Boundary Conditions
    if boundary_condition == "periodic":
        psi_plus[-1] = (psi_present[0]-psi_present[-1])/dx
        psi_minus[0] = (psi_present[0]-psi_present[-1])/dx
    else:
        print("Chosen boundary condition not yet implemented!")
        
    # Update according to advection equation
    psi_future = psi_present - dt*(u_minus*psi_minus + u_plus*psi_plus)
    
    return psi_future

# Define the domain length, number of grid points, and grid spacing
nx = 300
dx = L/(nx-1)

# Set the flow to be easterly
u = 0.5

# Create the x grid
x = np.linspace(-L/2., L/2., nx)

# Create the intial disturbance
psi = np.exp(-(x)**2)

# Set the time step in accordance with the CFL criteria
dt = 0.1*dx/abs(u)
print(f"Our stable choice of dt is: {round(dt,5)}")

# Specify the number of time steps
nt = 300

# Set up the initial conditions
psi_future = copy_functions(psi)
psi_future[-1] = psi_future[0]

# Plot the initial conditions
plot_function(x,psi_future,0.,u,0)

# Loop through times and update the wave function
for n in range(1,nt+1):
    
    # Advance n to n+1
    psi_present = copy_functions(psi_future)
        
    # Update with advection
    psi_future = downwind_advection(psi_future,psi_present,u,dt,nx,dx)
    
    # PLot the function
    if n in [20,40,60,80,100,200,300]:
        plot_function(x,psi_future,n*dt,u,0)

What if we decrease $\Delta x$? Will that change the stability? Recall that, from the CFL condition, we're keeping $\Delta t$ proportional to $\Delta x$, so our timestep will also decrease. Never the less, let's still choose the same number of time steps, in case the model begins to blow up. So we'll step through until only about 0.4 seconds, meaning the wave will only shift about 0.2 meters if it works properly.

In [ ]:
# Define the domain length, number of grid points, and grid spacing
nx = 3000
dx = L/(nx-1)

# Set the flow to be easterly
u = 0.5

# Create the x grid
x = np.linspace(-L/2., L/2., nx)

# Create the intial disturbance
psi = np.exp(-(x)**2)

# Set the time step in accordance with the CFL criteria
dt = 0.1*dx/abs(u)
print(f"Our stable choice of dt is: {round(dt,5)}")

# Specify the number of time steps
nt = 300

# Set up the initial conditions
psi_future = copy_functions(psi)
psi_future[-1] = psi_future[0]

# Plot the initial conditions
plot_function(x,psi_future,0.,u,0)

# Loop through times and update the wave function
for n in range(1,nt+1):
    
    # Advance n to n+1
    psi_present = copy_functions(psi_future)
        
    # Update with advection
    psi_future = downwind_advection(psi_future,psi_present,u,dt,nx,dx)
    
    # PLot the function
    if n in [20,40,60,80,100,200,300]:
        plot_function(x,psi_future,n*dt,u,0)

So reducing the gridspacing did not help. As mentioned before, the downwind scheme is unconditionally unstable, meaning that it will break no matter what we choose. So here's the point of the need for an upwind scheme. What if we had a flow that, rather than being constant, varied from positive values to negative values in $x$? We would need to be careful to never implement a backwind scheme over any gridpoint. This means we'd need select whether to perform backwards or forward differencing in space depending entirely on the value of $u$ at each grid point. For now, we're assuming constant flow, but we'll explore this more in the next section.

## CFL Condition Breaking

Before we move on, let's look at two more examples:
1. Choose a timestep such that

   $$ \Big| \frac{u \Delta t}{\Delta x} \Big| = 1 .$$
   
   That is, we need to choose
   
   $$ \Delta t = \frac{\Delta x}{|u|} .$$
   
2. Choose a timestep such that

   $$ \Big| \frac{u \Delta t}{\Delta x} \Big| = 3 .$$
   
   That is, we need to choose
   
   $$ \Delta t = \frac{3 \Delta x}{|u|} .$$
   
### Option 1

Let's implement the model with

$$ \Delta t = \frac{\Delta x}{|u|} .$$

I'll also change the thickness of the numerical solution for reasons that will become apparent shortly.

In [ ]:
def plot_function(x,psi,time,u,x0):
    
    '''Plot the function for time t'''

    # Plot function
    fig,ax = plt.subplots(figsize=(12,6));
    ax.plot(x,psi_future,label="Predicted",linewidth=2.5);
    ax.plot(x,np.exp(-(x-x0-((u*time)))**2),label="Actual")
    ax.set_xlabel("$x$",fontsize=14);
    ax.set_ylabel("$\psi$",fontsize=14);
    ax.set_title(f"t = {round(time,1)}",fontsize=18);
    ax.set_xlim(min(x),max(x))
    ax.legend(fontsize=14,loc='upper right')

# Define the domain length, number of grid points, and grid spacing
nx = 300
dx = L/(nx-1)

# Create the x grid
x = np.linspace(-L/2., L/2., nx)

# Create the intial disturbance
psi = np.exp(-(x)**2)

# Set the time step in accordance with the CFL criteria
dt = dx/u
print(f"Our stable choice of dt is: {round(dt,5)}")

# Stop after ~20 seconds of advecting the wave
stop_time = 10.
nt = round(stop_time/dt)+1

# Set up the initial conditions
psi_future = copy_functions(psi)
psi_future[-1] = psi_future[0]

# Plot the initial conditions
plot_function(x,psi_future,0.,u,0)

# Loop through times and update the wave function
for n in range(1,nt+1):
    
    # Advance n to n+1
    psi_present = copy_functions(psi_future)
        
    # Update with advection
    psi_future = upwind_advection(psi_future,psi_present,u,dt,nx,dx)
    
    # PLot the function
    if n % round(nt/5) == 0:
        plot_function(x,psi_future,n*dt,u,0)

So this time, the model works really well! By $t=10$ seconds, there is no apparent dampening or amplification of the wave, which is nice. So why not start with this? Well, the CFL criteria is quite simple for a constant flow advection model like this one, but what the criteria really states is that 

$$ \Big| \frac{c \Delta t}{\Delta x} \Big| \leq 1 $$

where $c$ is the *maximum* speed of a wave resolved in the model. Here, we have one disturbance moving no other speed than $|u|$, so it's straightforward. Some models can allow for other types of waves that we can't predict so obviously, so even if we have a good guess that fastest waves in the model will move at speed $c$, we may need to choose an even smaller timestep to ensure nothing ever causes the model to become unstable if the criteria is violated. Let's have a look at a case where it is now violated.

### Option 2

Let's now implement the model with the timestep defined by

$$ \Delta t = \frac{3 \Delta x}{|u|} .$$

In [ ]:
def plot_function(x,psi,time,u,x0):
    
    '''Plot the function for time t'''

    # Plot function
    fig,ax = plt.subplots(figsize=(12,6));
    ax.plot(x,psi_future,label="Predicted");
    ax.plot(x,np.exp(-(x-x0-((u*time)))**2),label="Actual")
    ax.set_xlabel("$x$",fontsize=14);
    ax.set_ylabel("$\psi$",fontsize=14);
    ax.set_title(f"t = {round(time,1)}",fontsize=18);
    ax.set_xlim(min(x),max(x))
    ax.legend(fontsize=14,loc='upper right')

# Define the domain length, number of grid points, and grid spacing
nx = 300
dx = L/(nx-1)

# Create the x grid
x = np.linspace(-L/2., L/2., nx)

# Create the intial disturbance
psi = np.exp(-(x)**2)

# Set the time step in accordance with the CFL criteria
dt = 3*dx/u
print(f"Our stable choice of dt is: {round(dt,5)}")

# Stop after ~20 seconds of advecting the wave
stop_time = 10.
nt = round(stop_time/dt)+1

# Set up the initial conditions
psi_future = copy_functions(psi)
psi_future[-1] = psi_future[0]

# Plot the initial conditions
plot_function(x,psi_future,0.,u,0)

# Loop through times and update the wave function
for n in range(1,nt+1):
    
    # Advance n to n+1
    psi_present = copy_functions(psi_future)
        
    # Update with advection
    psi_future = upwind_advection(psi_future,psi_present,u,dt,nx,dx)
    
    # PLot the function
    if n % round(nt/5) == 0:
        plot_function(x,psi_future,n*dt,u,0)

And now we're back to an unstable scenario. We've broken the CFL criteria, and the model is now unstable. So our upstream scheme is *conditionally stable* under the restriction of the CFL condition. 